## General steps

###**Import TFRS**

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.3 MB/s eta 0:00:00


In [ ]:
from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
# Additional for basic retrieval
!pip install -q scann

In [ ]:
# Additional for basic retrieval
import pprint
import tempfile

###**The datasets**

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [ ]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [ ]:
dataset_name = "movielens/100k-ratings"
dataset, info = tfds.load(dataset_name, split="train", with_info=True)
print(info.features)

In [ ]:
dataset_name = "movielens/100k-movies"
dataset, info = tfds.load(dataset_name, split="train", with_info=True)
print(info.features)

##Quickstart
[TensorFlow Recommenders: Quickstart](https://www.tensorflow.org/recommenders/examples/quickstart)

Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

###**Define a model**

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [ ]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)



Define the two models and the retrieval task.

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

###**Fit and evaluate it.**

Create the model, train it, and generate predictions:

In [ ]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

##**Applying quickstart to our own dataset**

###Import module and connect to gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd /content/gdrive/My Drive/Colab_Notebooks/Bangkit_Capstone/Dataset

/content/gdrive/My Drive/Colab_Notebooks/Bangkit_Capstone/Dataset


In [ ]:
df = pd.read_csv("last_merged_data.csv")

In [ ]:
# Print the row and column numbers
num_rows, num_columns = df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 6013
Number of columns: 10


In [ ]:
df.head()

,gunungID,nama_gunung,ketinggian,kota,provinsi_gunung,provinsi_user,userID,name,keahlian,tingkat_kesulitan
0,1,Arjuno,3339,Malang,Jawa Timur,Jawa Timur,43,Everleigh Cain,Pemula,Sulit
1,1,Arjuno,3339,Malang,Jawa Timur,Jawa Timur,133,Elise Winters,Pemula,Sulit
2,1,Arjuno,3339,Malang,Jawa Timur,Jawa Timur,464,Thomas Farley,Sedang,Sulit
3,1,Arjuno,3339,Malang,Jawa Timur,Jawa Timur,483,Milana Wu,Sedang,Sulit
4,1,Arjuno,3339,Malang,Jawa Timur,Jawa Timur,495,Genevieve Gillespie,Pemula,Sulit


In [ ]:
df["gunungID"] = df["gunungID"].astype("string")
df["nama_gunung"] = df["nama_gunung"].astype("string")
df["ketinggian"] = df["ketinggian"].astype("int64")
df["kota"] = df["kota"].astype("string")
df["provinsi_gunung"] = df["provinsi_gunung"].astype("string")
df["tingkat_kesulitan"] = df["tingkat_kesulitan"].astype("string")
df["userID"] = df["userID"].astype("string")
df["name"] = df["name"].astype("string")
df["keahlian"] = df["keahlian"].astype("string")
df["provinsi_user"] = df["provinsi_user"].astype("string")

In [ ]:
# mountain_data = pd.read_csv("Gunung_fix.csv")
# user_data = pd.read_csv("user_sample.csv")

In [ ]:
# Converting separated data to tfds
# mountain_tf_dataset = tf.data.Dataset.from_tensor_slices(dict(mountain_data))
# user_tf_dataset = tf.data.Dataset.from_tensor_slices(dict(user_data))

In [ ]:
# Converting merged dataset to tfds
df_tf_dataset = tf.data.Dataset.from_tensor_slices(dict(df))

In [ ]:
# Get column data types
merged_data_types = {key: value.dtype for key, value in df_tf_dataset.element_spec.items()}
for key, value in merged_data_types.items():
    print(key, value)

gunungID <dtype: 'string'>
nama_gunung <dtype: 'string'>
ketinggian <dtype: 'int64'>
kota <dtype: 'string'>
provinsi_gunung <dtype: 'string'>
provinsi_user <dtype: 'string'>
userID <dtype: 'string'>
name <dtype: 'string'>
keahlian <dtype: 'string'>
tingkat_kesulitan <dtype: 'string'>


In [ ]:
# Get column data types
# mountain_data_types = {key: value.dtype for key, value in mountain_tf_dataset.element_spec.items()}
# print(mountain_data_types)

# Get column data types
# user_data_types = {key: value.dtype for key, value in user_tf_dataset.element_spec.items()}
# print(user_data_types)

In [ ]:
# Select the basic features.
climbable = df_tf_dataset.map(lambda x: {
    "nama_gunung": x["nama_gunung"],
    "userID": x["userID"]
})
mount = df_tf_dataset.map(lambda x: x["nama_gunung"])

In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(climbable.map(lambda x: x["userID"]))

mount_names_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
mount_names_vocabulary.adapt(mount)

In [ ]:
class MountainModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      mountain_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.mountain_model = mountain_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["userID"])
    mountain_embeddings = self.mountain_model(features["nama_gunung"])

    return self.task(user_embeddings, mountain_embeddings)

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
mountain_model = tf.keras.Sequential([
    mount_names_vocabulary,
    tf.keras.layers.Embedding(mount_names_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    mount.batch(128).map(mountain_model)
  )
)

In [ ]:
# Create a retrieval model.
model = MountainModel(user_model, mountain_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
# model.fit(climbable.batch(4096), epochs=3)
model.fit(climbable.batch(8192), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    mount.batch(100).map(lambda title: (title, model.mountain_model(title))))

Epoch 1/3
1/1 [==============================] - 6s 6s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 52323.5078 - regularization_loss: 1.9523 - total_loss: 52325.4609
Epoch 2/3
1/1 [==============================] - 4s 4s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 51579.6602 - regularization_loss: 13.5224 - total_loss: 51593.1836
Epoch 3/3
1/1 [==============================] - 7s 7s/step - factorized_top_k/top_1_categorical_accuracy: 0.8073 - factorized_top_k/top_5_cate

In [ ]:
# Get some recommendations.
_, titles = index(np.array(["43"]))
print(f"Top 3 recommendations for user 43: {titles[0, :3]}")

Top 3 recommendations for user 43: [b'Kawi' b'Kawi' b'Kawi']


In [ ]:
# Get some recommendations.
_, titles = index(np.array(["43"]))
print(f"Top recommendations for user 43: {titles[0, :1]}")

Top recommendations for user 43: [b'Kawi']


In [ ]:
duplicates = df[df.duplicated('name')]
# print(duplicates)
duplicates

,gunungID,nama_gunung,ketinggian,kota,provinsi_gunung,provinsi_user,userID,name,keahlian,tingkat_kesulitan
63,8,Kawi,2551,Malang,Jawa Timur,Jawa Timur,43,Everleigh Cain,Pemula,Sulit
64,8,Kawi,2551,Malang,Jawa Timur,Jawa Timur,133,Elise Winters,Pemula,Sulit
65,8,Kawi,2551,Malang,Jawa Timur,Jawa Timur,464,Thomas Farley,Sedang,Sulit
66,8,Kawi,2551,Malang,Jawa Timur,Jawa Timur,483,Milana Wu,Sedang,Sulit
67,8,Kawi,2551,Malang,Jawa Timur,Jawa Timur,495,Genevieve Gillespie,Pemula,Sulit
...,...,...,...,...,...,...,...,...,...,...
6008,248,Pantai Siombak,3700,Tolikara,Papua,Papua,4542,Jovanni Case,Pemula,Sulit
6009,248,Pantai Siombak,3700,Tolikara,Papua,Papua,4583,Mavis Reyna,Mahir,Sulit
6010,248,Pantai Siombak,3700,Tolikara,Papua,Papua,4743,Bridget Prince,Mahir,Sulit
6011,248,Pantai Siombak,3700,Tolikara,Papua,Papua,4817,Maryam Sweeney,Mahir,Sulit


### Trying some configuration

In [ ]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(
        user_ids_vocabulary.vocabulary_size(),
        64,
        embeddings_regularizer=tf.keras.regularizers.l2(0.01)
    )
])

mountain_model = tf.keras.Sequential([
    mount_names_vocabulary,
    tf.keras.layers.Embedding(
        mount_names_vocabulary.vocabulary_size(),
        64,
        embeddings_regularizer=tf.keras.regularizers.l2(0.01)
    )
])
